# Bandit Experiments

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.primitives import Sampler
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector
from qiskit.circuit.classical import expr
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakePerth
from qiskit_ibm_runtime import SamplerV2
from qiskit import transpile
from qiskit.visualization import plot_histogram, plot_error_map, plot_distribution
from qiskit.result.mitigation import base_readout_mitigator

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import seaborn as sns
sns.set_style('dark')

In [ ]:
# HELPER FUNCTIONS

def generate_random_states(n=1000):
    random_list = [np.random.uniform(size=n)*pow(1j, (i%2)) for i in range(8)]
    random_state = []
    for i in range(0, 8, 2):
        random_state.append(random_list[i] + random_list[i+1])
    random_state = np.array(random_state).T
    for idx, row in enumerate(random_state):
        # row /= np.sum(np.abs(row))
        row /= np.sqrt(pow(np.abs(row), 2).sum())

    return random_state

def generate_product_states(n=1000):
    theta1 = np.random.uniform(0, 2*np.pi, size=n)
    theta2 = np.random.uniform(0, 2*np.pi, size=n)
    phi1 = np.random.uniform(0, np.pi, size=n)
    phi2 = np.random.uniform(0, np.pi, size=n)
    random_state = [np.cos(theta1)*np.cos(theta2), np.cos(theta1)*np.exp(1j*phi2)*np.sin(theta2), np.exp(1j*phi1)*np.sin(theta1)*np.cos(theta2), np.exp(1j*(phi1+phi2))*np.sin(theta1)*np.sin(theta2)]
    return np.array(random_state).T

def get_k(state):
    a, b, c, d = state
    r = np.abs(a*d-b*c).item()**2
    if r < 1e-30:
        return 0
    return r

def post_process_bitstrings(bitstrings):
    res = np.zeros(len(bitstrings))
    for idx, bitstr in enumerate(bitstrings):
        if bitstr in ['1 0 0 1', '1001', '0 0 1 1', '0011']:
            res[idx] = 1
    return res

def von_neuman_from_beta(beta):
    t = np.sqrt(1-4*beta)
    eigs = [0.5*(1+t), 0.5*(1-t)]
    s = 0
    for eig in eigs:
        if eig!=0 or eig!=1:
            s += -eig*np.log2(eig)

    return s

In [ ]:
np.random.seed(20)

k = generate_random_states(1000)
w = generate_product_states(1000)

### LOCC Circuit Builder

In [ ]:
def build_locc_circuit(initial_state):
    c1 = ClassicalRegister(1)
    c2 = ClassicalRegister(1)
    c3 = ClassicalRegister(1)
    c4 = ClassicalRegister(1)
    # cbits = ClassicalRegister(4, 'meas')
    qbits = QuantumRegister(4)
    
    qc = QuantumCircuit(qbits, c1, c2, c3, c4)
    
    initial_state = Statevector(initial_state)
    tensor_state = initial_state^initial_state
    qc.initialize(tensor_state) # to initialize with any state
    
    # qc.x(0)
    # # qc.x(1)
    # qc.x(2)
    # # qc.x(3)
    
    qc.barrier()
    qc.measure(1, 1)
    qc.measure(3, 3)
    qc.barrier()
    # qc.cx(0, 2)
    # qc.cx(0, 2)
    
    with qc.if_test(expr.equal(expr.bit_xor(c2, c4), 1)):
        # qc.x(0)
        qc.cx(0, 2)
        qc.cx(2, 0)
        # qc.ry(np.pi/4, 1)   #
        # qc.ccx(3, 2, 1)     # (For cch gate)   
        # qc.ry(-np.pi/4, 1)  #
        qc.cz(2, 0)
        qc.ch(2, 0)
        qc.cx(2, 0)
        qc.cx(0, 2)
        
    # qc.cx(0, 2)
    qc.barrier()
    # qc.measure(0, 0)
    qc.measure(0, 0)
    # qc.measure(2, 2)
    qc.measure(2, 2)

    return qc

## (m, K) Multi-armed bandits problem

We have **K** arms out of which **m** have distillable entanglement greater than some threshold $\beta$. We implement the *lil-HDoC* algorithm the address the Threshold Bandit Problem (TBP).

We generate random states by sampling the complex coefficients from a uniform distribution and then renormalizaing it

In [ ]:
np.random.seed(80) # For 5 arms
# np.random.seed(100) # For 7 arms

n_arms = 5
random_ids = np.random.choice(range(len(k)), size=n_arms)

In [ ]:
arms = k[random_ids]

In [ ]:
print(arms)

In [ ]:
val = np.array([get_k(arm) for arm in arms])
print("The Values of beta are : ", val)
print("Average Value of beta : ", np.mean(val))
print("Standard Deviation of beta : ", np.std(val))

In [ ]:
arms_idx = set(list(range(len(arms))))

In [ ]:
start = time.time()
n = 1000000
bitstrings = []
aer_sim = AerSimulator()

for idx, arm in enumerate(arms):
    circuit = build_locc_circuit(arm)
    pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1, seed_transpiler=20)
    isa_qc = pm.run(circuit)
    job = aer_sim.run([isa_qc], shots=n, seed_simulator=20, memory=True)
    result = job.result()
    bitstrs = result.get_memory()
    bitstrings.append(post_process_bitstrings(bitstrs))

bitstrings = np.array(bitstrings).T

end = time.time()
print(end-start)

#### Threshold Bandit Problem (TBP)

In [ ]:
def U_bound(eps, t, w):
    return (1+np.sqrt(eps))*np.sqrt(2*(1/4)*(1+eps)*np.log(np.log((1+eps)*t)/w)/t)

def lilHDoC(bitstrings, n_arms,  arms_idx, error_prob=0.01, threshold=0.02, bound_method='lil'):
    np.random.seed(20)
    estimated_mean = np.zeros(n_arms)
    iter_idx = np.zeros(n_arms).astype(int)
    max_iters = 10000000
    prev_arm = None
    good_arms = []
    bad_arms = []

    # Params
    b = n_arms + 1
    c = max(1/error_prob, np.e)
    eps = np.linspace(0, 1, endpoint=True, num=1000000)
    r = (1+np.sqrt(eps))**2*(1+eps)
    eps_prime = eps[np.argmax(r[r<=1+min(np.log(np.log(b))/np.log(b), np.log(np.log(c))/np.log(c))])]
    # print(f"Eps Prime : {eps_prime}")
    r = (1+np.sqrt(eps_prime))**2*(1+eps_prime)
    # print(f"R : {r}")

    c_eps = (2 + eps_prime)/eps_prime * pow(1/np.log(1 + eps_prime), 1 + eps_prime)
    # print(f"C_Eps : {c_eps}")
    thres = 0.25*(pow(n_arms/error_prob, r-1)*pow(c_eps, r))
    # print(f'Thres : {thres}')
    
    # Binary Search for T
    for t in range(1, 100000):
        if t*t/pow(np.log((1+eps_prime)*t), r) >= thres:
            T = t
            # print(f"T : {T}")
            break

    delta = abs(val - threshold)
    up_lim = 2*((1+eps_prime)*(1+np.sqrt(eps_prime))**2)/(delta*delta)*np.log(2*c_eps*n_arms*np.log(2*c_eps*n_arms*pow((1+np.sqrt(eps_prime))*(1+eps_prime), 2)/(error_prob*delta*delta))/error_prob)
    
    for idx in arms_idx:
        estimated_mean = np.mean(bitstrings[:T, :], axis=0)
        iter_idx = np.ones(n_arms, dtype=int)*(T)

    iteration = n_arms*T + 1
    while len(arms_idx) > 0 and iteration < max_iters:
        # if iteration % 100000 == 0:
            # print(f'Iteration : {iteration}, Good Arms : {good_arms}, Bad Arms : {bad_arms}, Iters : {iter_idx}', end='\r')
        
        ids = list(arms_idx)
        u_t = np.sqrt(np.log(iteration)/(2*iter_idx[ids]))
        # arm = ids[np.argmax(estimated_mean[ids] + u_t)]
        b = estimated_mean[ids] + u_t
        arm = ids[np.random.choice(np.where(b == b.max())[0])]

        # Pulling arm
        # print(iter_idx[arm], arm)
        sample = bitstrings[iter_idx[arm], arm]
        estimated_mean[arm] = (estimated_mean[arm]*iter_idx[arm] + sample)/(iter_idx[arm] + 1)
        iter_idx[arm] += 1

        # Elimination
        if bound_method =='lil':
            bound = U_bound(eps_prime, iter_idx[arm], error_prob/(c_eps*n_arms))
        else:
            bound = np.sqrt(np.log(1/error_prob)/(2*(iter_idx[arm]+1)))

        if estimated_mean[arm] - bound >= threshold:
            good_arms.append(arm)
            arms_idx.remove(arm)
        elif estimated_mean[arm] + bound < threshold:
            bad_arms.append(arm)
            arms_idx.remove(arm)

        iteration += 1

    return good_arms, bad_arms, iteration, T, np.sum(up_lim)

In [ ]:
good, bad, stop, T = lilHDoC(bitstrings, len(arms_idx), arms_idx.copy(), error_prob=0.001, threshold=0.05)

In [ ]:
print("GOOD ARMS     : ", val[good])
print("BAD ARMS      : ", val[bad])
print("STOPPING TIME : ", stop)

#### Relationship b/w Stopping Time and Confidence $\delta$

The experiments are ensembled over 10 runs

In [ ]:
conf = np.logspace(-3, -1, endpoint=True, num=10)
stopping_times_1 = []

n = 10000000
bitstrings = []
aer_sim = AerSimulator()

for idx, arm in enumerate(arms):
    circuit = build_locc_circuit(arm)
    pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1, seed_transpiler=20)
    isa_qc = pm.run(circuit)
    job = aer_sim.run([isa_qc], shots=n, seed_simulator=20, memory=True)
    result = job.result()
    bitstrs = result.get_memory()
    bitstrings.append(post_process_bitstrings(bitstrs))

bitstrings = np.array(bitstrings).T

start = time.time()
for i in range(10):
    u_lims = []
    stops = []
    for c in conf:
        print(f"Iteration : {i+1}, Confidence : {c}", end='\r')
        good, bad, stop, T, u_lim = lilHDoC(bitstrings[i*1000000:(i+1)*1000000, :], len(arms_idx), arms_idx.copy(), error_prob=c, threshold=0.05, bound_method='lil')
        stops.append(stop)
        u_lims.append(u_lim)
        
    stopping_times_1.append(stops)

In [ ]:
stopping_times_1 = np.array(stopping_times_1)

In [ ]:
plt.errorbar(np.log10(conf), 2*np.mean(stopping_times_1, axis=0), yerr=2*np.std(stopping_times_1, axis=0), capsize=4)
plt.grid()
plt.grid(which='minor', ls=':', lw=0.5)
plt.minorticks_on()
plt.xlabel("Confidence ($\delta$)", labelpad=5, fontsize=13)
plt.ylabel("Copy Complexity", labelpad=5, fontsize=13)
plt.title("Copy Complexity vs Confidence ($\delta$)")
plt.show()